In [1]:
seed = 1
sparsity = 0.8
width = 32

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import torch
import torch.nn as nn
import os
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import time
import copy
import sys

import random
import numpy as np
import torch
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import scipy.stats as ss
from timm.data import Mixup
from timm.loss import SoftTargetCrossEntropy
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable

import sys
import numpy as np
import torch.nn.utils.prune as prune
from datautils import *

In [4]:
def random_seed(seed=42, rank=0):
    torch.manual_seed(seed + rank)
    np.random.seed(seed + rank)
    random.seed(seed + rank)

random_seed(seed)

In [5]:
train_loader, val_loader = get_loaders(
    "imagenet", path="",
    batchsize=256, workers=8,
    nsamples=-1, seed=0,
    noaug=False
)

In [6]:
def train(train_loader, model, criterion, optimizer, scaler, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    #top1 = AverageMeter('Acc@1', ':6.2f')
    #top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)

        # compute output
        with torch.cuda.amp.autocast():
            output = model(images)
            loss = criterion(output, target)

        # measure accuracy and record loss
        #acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        #top1.update(acc1[0], images.size(0))
        #top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        #loss.backward()
        #optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 50 == 0:
            progress.display(i)
        if epoch == -1 and i == 50:
            break

    return losses.avg


def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            images = images.cuda(non_blocking=True)
            target = target.cuda(non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 50 == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        #print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
        #      .format(top1=top1, top5=top5))

    return top1.avg

In [7]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [8]:
def get_res(epochs=20):
    from torchvision.models import resnet50
    model = resnet50(pretrained=True)
    #model = resnet20()
    #model.load_state_dict(torch.load("done_rn20_%s_%s_160_amp_fixed.pth" % (seed, width)))
    print(model, file=sys.stderr)
    model.cuda()
    
    
    #optimizer = torch.optim.AdamW(model.parameters(), 0.001)
    optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum=0.9, nesterov=True, weight_decay=1e-4)
    opt0 = torch.optim.SGD(model.parameters(), 0.0, momentum=0.9, nesterov=True, weight_decay=1e-4)
    #scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2], gamma=0.1)
    scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer, total_iters=epochs, power=1)
    #scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 0.004, epochs, cycle_momentum=False)
    criterion = nn.CrossEntropyLoss()#SoftTargetCrossEntropy()
    criterion_val = nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler()
    
    total_params = 0
    for n, m in model.named_modules():
        if ("conv" in n or "downsample" in n) and "Conv" in str(type(m)) and m.weight.shape[1] > 3:
            print(n, m.weight.shape)
            total_params += m.weight.numel()
            prune.l1_unstructured(m, name='weight', amount=sparsity)
    print("tot", total_params)
    
    best_acc1 = 0
    acc1 = validate(val_loader, model, criterion_val).item()
    print("start acc no bn", acc1)
    train_loss = train(train_loader, model, criterion, opt0, scaler, -1)
    acc1 = validate(val_loader, model, criterion_val).item()
    total_active = 0
    for n, m in model.named_modules():
        if "conv" in n or "downsample" in n and "Conv" in str(type(m)) and m.weight.shape[1] > 3:
            total_active += (m.weight != 0).sum().item()
    print("start acc bn", acc1, total_active, total_active / total_params)

    for epoch in range(epochs):
        train_loss = train(train_loader, model, criterion, optimizer, scaler, epoch)
        acc1 = validate(val_loader, model, criterion_val).item()
        scheduler.step()
        
        # remember best acc@1 and save checkpoint
        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1)
        total_active = 0
        for n, m in model.named_modules():
            if "conv" in n or "downsample" in n and "Conv" in str(type(m)) and m.weight.shape[1] > 3:
                total_active += (m.weight != 0).sum().item()
        print("epoch", epoch, train_loss, acc1, optimizer.param_groups[0]['lr'], total_active, total_active / total_params)
    
    return acc1, copy.deepcopy(model.state_dict())

acc, end = get_res()

print("acc", acc)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

layer1.0.conv1 torch.Size([64, 64, 1, 1])
layer1.0.conv2 torch.Size([64, 64, 3, 3])
layer1.0.conv3 torch.Size([256, 64, 1, 1])
layer1.0.downsample.0 torch.Size([256, 64, 1, 1])
layer1.1.conv1 torch.Size([64, 256, 1, 1])
layer1.1.conv2 torch.Size([64, 64, 3, 3])
layer1.1.conv3 torch.Size([256, 64, 1, 1])
layer1.2.conv1 torch.Size([64, 256, 1, 1])
layer1.2.conv2 torch.Size([64, 64, 3, 3])
layer1.2.conv3 torch.Size([256, 64, 1, 1])
layer2.0.conv1 torch.Size([128, 256, 1, 1])
layer2.0.conv2 torch.Size([128, 128, 3, 3])
layer2.0.conv3 torch.Size([512, 128, 1, 1])
layer2.0.downsample.0 torch.Size([512, 256, 1, 1])
layer2.1.conv1 torch.Size([128, 512, 1, 1])
layer2.1.conv2 torch.Size([128, 128, 3, 3])
layer2.1.conv3 torch.Size([512, 128, 1, 1])
layer2.2.conv1 torch.Size([128, 512, 1, 1])
layer2.2.conv2 torch.Size([128, 128, 3, 3])
layer2.2.conv3 torch.Size([512, 128, 1, 1])
layer2.3.conv1 torch.Size([128, 512, 1, 1])
layer2.3.conv2 torch.Size([128, 128, 3, 3])
layer2.3.conv3 torch.Size([512, 

Epoch: [0][2550/5005]	Time  0.558 ( 0.559)	Data  0.000 ( 0.001)	Loss 1.0736e+00 (1.1638e+00)
Epoch: [0][2600/5005]	Time  0.558 ( 0.559)	Data  0.000 ( 0.001)	Loss 1.0988e+00 (1.1632e+00)
Epoch: [0][2650/5005]	Time  0.558 ( 0.559)	Data  0.000 ( 0.001)	Loss 9.0896e-01 (1.1628e+00)
Epoch: [0][2700/5005]	Time  0.558 ( 0.559)	Data  0.000 ( 0.001)	Loss 1.0670e+00 (1.1624e+00)
Epoch: [0][2750/5005]	Time  0.559 ( 0.559)	Data  0.000 ( 0.001)	Loss 1.2468e+00 (1.1619e+00)
Epoch: [0][2800/5005]	Time  0.559 ( 0.559)	Data  0.000 ( 0.001)	Loss 1.0274e+00 (1.1613e+00)
Epoch: [0][2850/5005]	Time  0.558 ( 0.559)	Data  0.000 ( 0.001)	Loss 1.0289e+00 (1.1611e+00)
Epoch: [0][2900/5005]	Time  0.559 ( 0.559)	Data  0.000 ( 0.001)	Loss 1.0493e+00 (1.1602e+00)
Epoch: [0][2950/5005]	Time  0.559 ( 0.559)	Data  0.000 ( 0.001)	Loss 1.0800e+00 (1.1596e+00)
Epoch: [0][3000/5005]	Time  0.559 ( 0.559)	Data  0.000 ( 0.001)	Loss 1.1059e+00 (1.1591e+00)
Epoch: [0][3050/5005]	Time  0.559 ( 0.559)	Data  0.000 ( 0.001)	Loss 9

Epoch: [1][1650/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.002)	Loss 1.1183e+00 (1.0667e+00)
Epoch: [1][1700/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.002)	Loss 1.1739e+00 (1.0671e+00)
Epoch: [1][1750/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.002)	Loss 1.2097e+00 (1.0676e+00)
Epoch: [1][1800/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.002)	Loss 1.2935e+00 (1.0683e+00)
Epoch: [1][1850/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.002)	Loss 1.2205e+00 (1.0682e+00)
Epoch: [1][1900/5005]	Time  0.559 ( 0.560)	Data  0.000 ( 0.002)	Loss 1.1254e+00 (1.0685e+00)
Epoch: [1][1950/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.002)	Loss 1.3243e+00 (1.0686e+00)
Epoch: [1][2000/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.001)	Loss 1.1900e+00 (1.0695e+00)
Epoch: [1][2050/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.001)	Loss 8.6963e-01 (1.0697e+00)
Epoch: [1][2100/5005]	Time  0.560 ( 0.560)	Data  0.000 ( 0.001)	Loss 1.0618e+00 (1.0693e+00)
Epoch: [1][2150/5005]	Time  0.559 ( 0.560)	Data  0.000 ( 0.001)	Loss 1

Epoch: [2][ 750/5005]	Time  0.558 ( 0.562)	Data  0.000 ( 0.004)	Loss 1.1033e+00 (1.0289e+00)
Epoch: [2][ 800/5005]	Time  0.559 ( 0.562)	Data  0.000 ( 0.003)	Loss 1.1044e+00 (1.0291e+00)
Epoch: [2][ 850/5005]	Time  0.558 ( 0.561)	Data  0.000 ( 0.003)	Loss 1.0012e+00 (1.0304e+00)
Epoch: [2][ 900/5005]	Time  0.558 ( 0.561)	Data  0.000 ( 0.003)	Loss 9.5912e-01 (1.0298e+00)
Epoch: [2][ 950/5005]	Time  0.558 ( 0.561)	Data  0.000 ( 0.003)	Loss 1.0624e+00 (1.0298e+00)
Epoch: [2][1000/5005]	Time  0.558 ( 0.561)	Data  0.000 ( 0.003)	Loss 1.0298e+00 (1.0300e+00)
Epoch: [2][1050/5005]	Time  0.559 ( 0.561)	Data  0.000 ( 0.003)	Loss 1.1333e+00 (1.0315e+00)
Epoch: [2][1100/5005]	Time  0.558 ( 0.561)	Data  0.000 ( 0.003)	Loss 8.9054e-01 (1.0306e+00)
Epoch: [2][1150/5005]	Time  0.558 ( 0.561)	Data  0.000 ( 0.002)	Loss 1.1295e+00 (1.0315e+00)
Epoch: [2][1200/5005]	Time  0.558 ( 0.561)	Data  0.000 ( 0.002)	Loss 1.1133e+00 (1.0315e+00)
Epoch: [2][1250/5005]	Time  0.559 ( 0.561)	Data  0.000 ( 0.002)	Loss 1

...

Epoch: [16][1050/5005]	Time  0.559 ( 0.562)	Data  0.000 ( 0.003)	Loss 9.6731e-01 (7.7894e-01)
Epoch: [16][1100/5005]	Time  0.558 ( 0.562)	Data  0.000 ( 0.002)	Loss 9.7354e-01 (7.7964e-01)
Epoch: [16][1150/5005]	Time  0.558 ( 0.561)	Data  0.000 ( 0.002)	Loss 8.5428e-01 (7.7994e-01)
Epoch: [16][1200/5005]	Time  0.558 ( 0.561)	Data  0.000 ( 0.002)	Loss 6.6317e-01 (7.8008e-01)
Epoch: [16][1250/5005]	Time  0.559 ( 0.561)	Data  0.000 ( 0.002)	Loss 5.9824e-01 (7.8026e-01)
Epoch: [16][1300/5005]	Time  0.559 ( 0.561)	Data  0.000 ( 0.002)	Loss 8.1940e-01 (7.7943e-01)
Epoch: [16][1350/5005]	Time  0.558 ( 0.561)	Data  0.000 ( 0.002)	Loss 8.6015e-01 (7.7989e-01)
Epoch: [16][1400/5005]	Time  0.558 ( 0.561)	Data  0.000 ( 0.002)	Loss 8.3941e-01 (7.7931e-01)
Epoch: [16][1450/5005]	Time  0.559 ( 0.561)	Data  0.000 ( 0.002)	Loss 5.4901e-01 (7.7970e-01)
Epoch: [16][1500/5005]	Time  0.558 ( 0.561)	Data  0.000 ( 0.002)	Loss 7.8743e-01 (7.7939e-01)
Epoch: [16][1550/5005]	Time  0.558 ( 0.561)	Data  0.000 ( 0.

Epoch: [17][ 100/5005]	Time  0.558 ( 0.581)	Data  0.000 ( 0.022)	Loss 6.8795e-01 (7.6921e-01)
Epoch: [17][ 150/5005]	Time  0.559 ( 0.573)	Data  0.000 ( 0.015)	Loss 6.5588e-01 (7.7128e-01)
Epoch: [17][ 200/5005]	Time  0.558 ( 0.570)	Data  0.000 ( 0.011)	Loss 8.2722e-01 (7.6364e-01)
Epoch: [17][ 250/5005]	Time  0.559 ( 0.567)	Data  0.000 ( 0.009)	Loss 8.4938e-01 (7.6351e-01)
Epoch: [17][ 300/5005]	Time  0.558 ( 0.566)	Data  0.000 ( 0.008)	Loss 8.8848e-01 (7.6511e-01)
Epoch: [17][ 350/5005]	Time  0.558 ( 0.565)	Data  0.000 ( 0.007)	Loss 8.0716e-01 (7.6455e-01)
Epoch: [17][ 400/5005]	Time  0.559 ( 0.564)	Data  0.000 ( 0.006)	Loss 7.3322e-01 (7.6700e-01)
Epoch: [17][ 450/5005]	Time  0.560 ( 0.563)	Data  0.000 ( 0.005)	Loss 7.9011e-01 (7.6739e-01)
Epoch: [17][ 500/5005]	Time  0.558 ( 0.563)	Data  0.000 ( 0.005)	Loss 7.3769e-01 (7.6815e-01)
Epoch: [17][ 550/5005]	Time  0.558 ( 0.563)	Data  0.000 ( 0.004)	Loss 6.8866e-01 (7.6809e-01)
Epoch: [17][ 600/5005]	Time  0.558 ( 0.563)	Data  0.000 ( 0.

Epoch: [17][4500/5005]	Time  0.558 ( 0.559)	Data  0.000 ( 0.001)	Loss 7.9351e-01 (7.6801e-01)
Epoch: [17][4550/5005]	Time  0.559 ( 0.559)	Data  0.000 ( 0.001)	Loss 7.7050e-01 (7.6780e-01)
Epoch: [17][4600/5005]	Time  0.558 ( 0.559)	Data  0.000 ( 0.001)	Loss 6.4788e-01 (7.6793e-01)
Epoch: [17][4650/5005]	Time  0.559 ( 0.559)	Data  0.000 ( 0.001)	Loss 7.8464e-01 (7.6803e-01)
Epoch: [17][4700/5005]	Time  0.558 ( 0.559)	Data  0.000 ( 0.001)	Loss 8.7276e-01 (7.6803e-01)
Epoch: [17][4750/5005]	Time  0.558 ( 0.559)	Data  0.000 ( 0.001)	Loss 7.8922e-01 (7.6820e-01)
Epoch: [17][4800/5005]	Time  0.559 ( 0.559)	Data  0.000 ( 0.001)	Loss 8.5940e-01 (7.6827e-01)
Epoch: [17][4850/5005]	Time  0.558 ( 0.559)	Data  0.000 ( 0.001)	Loss 7.9251e-01 (7.6839e-01)
Epoch: [17][4900/5005]	Time  0.558 ( 0.559)	Data  0.000 ( 0.001)	Loss 6.8060e-01 (7.6821e-01)
Epoch: [17][4950/5005]	Time  0.558 ( 0.559)	Data  0.000 ( 0.001)	Loss 7.6958e-01 (7.6836e-01)
Epoch: [17][5000/5005]	Time  0.558 ( 0.559)	Data  0.000 ( 0.

Epoch: [18][3550/5005]	Time  0.561 ( 0.560)	Data  0.000 ( 0.001)	Loss 7.4037e-01 (7.4643e-01)
Epoch: [18][3600/5005]	Time  0.560 ( 0.560)	Data  0.000 ( 0.001)	Loss 7.2831e-01 (7.4652e-01)
Epoch: [18][3650/5005]	Time  0.559 ( 0.560)	Data  0.000 ( 0.001)	Loss 7.8507e-01 (7.4655e-01)
Epoch: [18][3700/5005]	Time  0.559 ( 0.560)	Data  0.000 ( 0.001)	Loss 7.9964e-01 (7.4646e-01)
Epoch: [18][3750/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.001)	Loss 6.7636e-01 (7.4624e-01)
Epoch: [18][3800/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.001)	Loss 7.4982e-01 (7.4631e-01)
Epoch: [18][3850/5005]	Time  0.559 ( 0.560)	Data  0.000 ( 0.001)	Loss 8.5131e-01 (7.4610e-01)
Epoch: [18][3900/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.001)	Loss 8.3028e-01 (7.4616e-01)
Epoch: [18][3950/5005]	Time  0.559 ( 0.560)	Data  0.000 ( 0.001)	Loss 8.6968e-01 (7.4617e-01)
Epoch: [18][4000/5005]	Time  0.562 ( 0.560)	Data  0.000 ( 0.001)	Loss 8.1373e-01 (7.4607e-01)
Epoch: [18][4050/5005]	Time  0.559 ( 0.560)	Data  0.000 ( 0.

Epoch: [19][2600/5005]	Time  0.560 ( 0.560)	Data  0.000 ( 0.001)	Loss 7.8668e-01 (7.2699e-01)
Epoch: [19][2650/5005]	Time  0.561 ( 0.560)	Data  0.000 ( 0.001)	Loss 6.3903e-01 (7.2719e-01)
Epoch: [19][2700/5005]	Time  0.560 ( 0.560)	Data  0.000 ( 0.001)	Loss 6.9846e-01 (7.2707e-01)
Epoch: [19][2750/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.001)	Loss 7.9705e-01 (7.2705e-01)
Epoch: [19][2800/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.001)	Loss 8.6591e-01 (7.2694e-01)
Epoch: [19][2850/5005]	Time  0.559 ( 0.560)	Data  0.000 ( 0.001)	Loss 8.2229e-01 (7.2709e-01)
Epoch: [19][2900/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.001)	Loss 7.6299e-01 (7.2736e-01)
Epoch: [19][2950/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.001)	Loss 7.7997e-01 (7.2766e-01)
Epoch: [19][3000/5005]	Time  0.559 ( 0.560)	Data  0.000 ( 0.001)	Loss 6.3463e-01 (7.2728e-01)
Epoch: [19][3050/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.001)	Loss 7.3583e-01 (7.2742e-01)
Epoch: [19][3100/5005]	Time  0.558 ( 0.560)	Data  0.000 ( 0.